# **Sesión 2:** Calibración de Cámara 📷⚙️

## 0. Preparación

In [ ]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import os
import cv2
import copy
import glob

## **Apartado A: Calibración de cámara** (derecha e izquierda)

En este apartado se realiza la calibración de dos cámaras de un sistema estereoscópico. Para ello se trabajará con las imágenes de las carpetas ``left`` y ``right``. En primer lugar se trabajará con la carpeta ``left``. Posteriormente, deberá repetir el proceso con las imágenes en la carpeta ``right``. Ambas carpetas contienen imágenes con las que se calibrarán las cámaras. En ellas aparece el patrón de calibración en diferentes posiciones y orientaciones. Estas imágenes serán los datos de entrada.

Los pasos que deberá seguir para calibrar una cámara son:

1. Defina y ejecute el método para cargar imágenes ``load_images()``.
2. Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.
3. Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``.
4. Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)`` que proporcione las coordenadas 3D de las esquinas del patrón. El sistema de referencia utilizado deberá estar anclado al propio patrón.
5. Utilice ``cv2.calibrateCamera`` para obtener los parámetros de calibración para la cámara izquierda.

### **Tarea A.1:** Defina y ejecute el método para cargar imágenes ``load_images()``.

In [ ]:
def load_images(filenames: List) -> List:
    return [cv2.imread(filename) for filename in filenames]

In [ ]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = ["../data/" + item for item in os.listdir("../data/") if item.endswith(".jpg")]
imgs = load_images(imgs_path)

### **Tarea A.2:** Detecte las esquinas de los patrones usando ``cv2.findChessboardCorners()``. Refine las detecciones con ``cv2.cornerSubPix()``.

In [ ]:
# TODO Find corners with cv2.findChessboardCorners()
chessboard_shape = (7, 9)
corners = []
for img in imgs:
    ret, corner = cv2.findChessboardCorners(img, chessboard_shape)
    corners.append((ret, corner))

In [ ]:
corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (7, 9), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

### **Tarea A.3:** Compruebe que las detecciones son correctas dibujando los resultados con ``cv2.drawChessboardCorners()``

In [ ]:
imgs_copy = copy.deepcopy(imgs)

In [ ]:
# TODO Use cv2.drawChessboardCorners() to draw the cornes
list_with_corners = []
for i in range(len(corners)):
    list_with_corners.append(cv2.drawChessboardCorners(imgs_copy[i], patternSize=chessboard_shape, patternWasFound=corners[i][0], corners=corners[i][1]))


In [ ]:
def show_image(img, window_name="Image"):
    cv2.imshow(window_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def write_image(img, filename):
    cv2.imwrite(filename, img)

for image in list_with_corners:
    show_image(image)


### **Tarea A.4:** Defina y ejecute el método ``get_chessboard_points(chessboard_shape, dx, dy)``

In [ ]:
# TODO Design the method. It should return a np.array with np.float32 elements
def get_chessboard_points(chessboard_shape, dx, dy):
    rows, cols = chessboard_shape
    objp = np.zeros((rows * cols, 3), np.float32)
    for y in range(rows):
        for x in range(cols):
            objp[y * cols + x, 0] = x * dx
            objp[y * cols + x, 1] = y * dy
            objp[y * cols + x, 2] = 0
    return objp

In [ ]:
# TODO You need the points for every image, not just one (consider a list comprehension)
chessboard_points = get_chessboard_points((6, 8), 30, 30)

### **Tarea A.5:** Utilice ``cv2.calibrateCamera()`` para obtener los parámetros de calibración para la cámara izquierda

In [ ]:
# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

In [ ]:
# TODO
obj= np.array([chessboard_points]*18)
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints=obj, imagePoints=valid_corners, imageSize=imgs[0].shape[:2], cameraMatrix=np.zeros((3,3)), distCoeffs=np.zeros((1,4)))


# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

In [ ]:
# Print outputs
print("Intrinsics:\n", intrinsics)
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)

### **Pregunta A.1:** Repita el proceso (carga de imágenes, detección y comprobación de esquinas, etc.) para la cámara derecha.

In [ ]:
# TODO Build a list containing the paths of all images from the left camera
imgs_path = ["../data/right/" + item for item in os.listdir("../data/right/") if item.endswith(".jpg")]
imgs = load_images(imgs_path) 

# TODO Find corners with cv2.findChessboardCorners()
chessboard_shape = (8, 6)
corners = []
for img in imgs:
    ret, corner = cv2.findChessboardCorners(img, chessboard_shape)
    corners.append((ret, corner))

corners_copy = copy.deepcopy(corners)
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01)

# TODO To refine corner detections with cv2.cornerSubPix() you need to input grayscale images. Build a list containing grayscale images.
imgs_gray = [cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) for img in imgs]

corners_refined = [cv2.cornerSubPix(i, cor[1], (8, 6), (-1, -1), criteria) if cor[0] else [] for i, cor in zip(imgs_gray, corners_copy)]

imgs_copy = copy.deepcopy(imgs)

# TODO Use cv2.drawChessboardCorners() to draw the cornes
list_with_corners = []
for i in range(len(corners)):
    list_with_corners.append(cv2.drawChessboardCorners(imgs_copy[i], patternSize=chessboard_shape, patternWasFound=corners[i][0], corners=corners[i][1]))

for image in list_with_corners[:2]:
    show_image(image)

# TODO You need the points for every image, not just one (consider a list comprehension)
chessboard_points = get_chessboard_points((6, 8), 30, 30)

# Filter data and get only those with adequate detections
valid_corners = [cor[1] for cor in corners if cor[0]]
# Convert list to numpy array
valid_corners = np.asarray(valid_corners, dtype=np.float32)

# TODO
obj= np.array([chessboard_points]*18)
rms, intrinsics, dist_coeffs, rvecs, tvecs = cv2.calibrateCamera(objectPoints=obj, imagePoints=valid_corners, imageSize=imgs[0].shape[:2], cameraMatrix=np.zeros((3,3)), distCoeffs=np.zeros((1,4)))

# Obtain extrinsics
extrinsics = list(map(lambda rvec, tvec: np.hstack((cv2.Rodrigues(rvec)[0], tvec)), rvecs, tvecs))

# Print outputs
print("Intrinsics:\n", intrinsics)
print("Extrinsics:\n", extrinsics[0])
print("Distortion coefficients:\n", dist_coeffs)
print("Root mean squared reprojection error:\n", rms)


### **Pregunta A.2:** Diferencias entre cv2.findChessboardCorners() y cv2.cornerSubPix()

In [ ]:

out = []
for i in range(2):
    if corners[i][0] and len(corners_refined[i]) > 0:
        # Copias para dibujar
        vis_coarse  = imgs[i].copy()
        vis_refined = imgs[i].copy()

        # Dibujo original
        cv2.drawChessboardCorners(
            vis_coarse, chessboard_shape, corners[i][1], True
        )

        # Dibujo refinado
        cv2.drawChessboardCorners(
            vis_refined, chessboard_shape, corners_refined[i], True
        )

        # Mostrar las imágenes
        show_image(vis_coarse)
        show_image(vis_refined)

        # Métricas: desplazamiento medio y máximo (en píxeles)
        c0 = corners[i][1].reshape(-1, 2).astype(np.float32)
        c1 = corners_refined[i].reshape(-1, 2).astype(np.float32)
        d  = np.linalg.norm(c0 - c1, axis=1)
        out.append((i, float(d.mean()), float(d.max())))

# Métricas para cada imagen
for i, mean_shift, max_shift in out:
    print(f"[A.2] Img {i}: mean shift = {mean_shift:.3f} px | max shift = {max_shift:.3f} px")


### **Pregunta A.3:** Número mínimo de imágenes necesarias para calibrar.

In [ ]:
# TODO Homework
import matplotlib.pyplot as plt


# N total de imágenes válidas detectadas
n_total = len(valid_corners)
print(f"Total de imágenes válidas disponibles: {n_total}")

# Lista de tamaños de subconjunto (número de imágenes a usar)
subset_sizes = [3, 5, 7, 9, 11, 13, 15, n_total]

# Listas para guardar resultados
rms_values = []

# Calibrar con distintos tamaños de conjunto
for n in subset_sizes:
    # Seleccionar las primeras n imágenes válidas
    obj_subset = [chessboard_points] * n
    img_subset = valid_corners[:n]

    rms, K, D, rvecs, tvecs = cv2.calibrateCamera(
        objectPoints=obj_subset,
        imagePoints=img_subset,
        imageSize=imgs[0].shape[:2],
        cameraMatrix=np.zeros((3,3)),
        distCoeffs=np.zeros((1,4))
    )

    rms_values.append(rms)
    print(f"Usando {n} imágenes → RMS = {rms:.4f}")

# Grafico de Pareto
plt.figure(figsize=(6,4))
plt.plot(subset_sizes, rms_values, marker='o')
plt.title("Diagrama de Pareto: número de imágenes vs RMS")
plt.xlabel("Número de imágenes utilizadas")
plt.ylabel("Error RMS (px)")
plt.grid(True)
plt.show()


## **Apartado B: Corrección de distorsión** (ojo de pez)

En este apartado se trabajará en la corrección de la distorsión debido a lentes de ojo de pez. Primero se calibrará una cámara con este tipo de lente, utilizando las imágenes de la carpeta ``fisheye``. Posteriormente se utilizarán estos parámetros de calibración para corregir la distorsión de una de las imágenes de calibración.

Los pasos que deberá seguir para calibrar una cámara con distorsión de ojo de pez son:

1. Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``.
2. Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.
3. Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero.
4. Defina los argumentos para la función de calibración.
5. Calibre con ``cv2.fisheye.calibrate()``

### **Tarea B.1:** Reutilice el método ``load_images()`` para cargar las imágenes de la carpeta ``fisheye``

In [ ]:
# TODO Build a list containing the paths of all images from the fisheye camera and load images
fisheye_imgs_path = ["../data/fisheye/" + item for item in os.listdir("../data/fisheye/") if item.endswith(".jpg")]
fisheye_imgs = load_images(fisheye_imgs_path)

### **Tarea B.2:** Detecte las equinas procesando las imágenes con los métodos ``cv2.findChessboardCorners()`` y ``cv2.cornerSubPix()``.

In [ ]:
imgs_corners = []
# Parameters for cv2.cornerSubPix()
subpix_criteria = (cv2.TERM_CRITERIA_EPS+cv2.TERM_CRITERIA_MAX_ITER, 30, 0.1)
chessboard_shape = (7, 6)

#TODO Complete the required parts of the loop
for img in fisheye_imgs:
    # TODO parse arguments to cv2.findChessboardCorners()
    ret, corners = cv2.findChessboardCorners(image=img, patternSize=chessboard_shape)
    
    # TODO convert image to grayscale to use cv2.cornerSubPix()
    gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) 
    if ret:  # Solo si encontró todas las esquinas
        # Refinar posiciones a nivel subpíxel
        refined_corners = cv2.cornerSubPix(
            gray_img,
            corners,
            (3, 3),
            (-1, -1),
            subpix_criteria
        )
        imgs_corners.append(refined_corners)
    else:
        print("no")

### **Tarea B.3:** Reutilice la función ``get_chessboard_points()`` para obtener las coordenadas del tablero

In [ ]:
# TODO Define the chessboard dimensions and the lenght of the squares (in [mm])
chessboard_dims = (6, 7)
length = 0.03
# TODO You need the points for every image, not just one (consider a list comprehension)
fisheye_chessboard_points = [
    get_chessboard_points(chessboard_dims, length, length).astype(np.float64)[np.newaxis, :, :]
    for _ in imgs_corners
]

### **Tarea B.4:** Defina los argumentos para la calibración

In [ ]:
# Parameters for cv2.fisheye.calibrate()
calibration_flags = cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC+cv2.fisheye.CALIB_FIX_SKEW
intrinsics = np.zeros((3, 3))
distortion = np.zeros((4, 1))
rotations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]
traslations = [np.zeros((1, 1, 3), dtype=np.float64) for _ in imgs_corners]


### **Tarea B.5:** Calibración

In [ ]:
rms, _, _, _, _ = \
cv2.fisheye.calibrate(fisheye_chessboard_points, imgs_corners, gray_img.shape[::-1], intrinsics, distortion, rotations, traslations, calibration_flags, subpix_criteria)

In [ ]:
# Show intrinsic matrix and distortion coefficients values
print("Intrinsics:", intrinsics)
print("Distortion:", distortion)
print("RMSE:", rms)

### **Pregunta B.1:** Corrija la distorsión de las 2 primeras imágenes de la carpeta ``fisheye``

In [ ]:
# TODO Search in the documentation to define 'dim'
dim = (fisheye_imgs[0].shape[1], fisheye_imgs[0].shape[0])
map1, map2 = cv2.fisheye.initUndistortRectifyMap(intrinsics, distortion, np.eye(3), intrinsics, dim, cv2.CV_16SC2)

In [ ]:
# TODO Homework: correct distortion using cv2.remap()
print("Intrinsics (K):\n", intrinsics)
print("\nDistortion (D):\n", distortion)

# Función para undistort usando los mapas encontrados antes
def undistort_fisheye_image(img, map1, map2):
    # Usamos INTER_LINEAR, podríamos ussar INTER_CUBIC
    # BORDER_CONSTANT evita “arrastres” negros por fuera del campo útil
    return cv2.remap(img, map1, map2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# Generar y guardar las 2 primeras imágenes sin distorsión
def save_rgb_as_bgr(path, img_rgb):
    img_bgr = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, img_bgr)

undist_0 = undistort_fisheye_image(fisheye_imgs[0], map1, map2)
undist_1 = undistort_fisheye_image(fisheye_imgs[1], map1, map2)

# Guardamos a disco las fotos
save_rgb_as_bgr("fisheye_undist_0_C.png", undist_0)
save_rgb_as_bgr("fisheye_undist_1_C.png", undist_1)
